In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch
!pip install tqdm
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
df = pd.read_csv("processed_train.csv")

In [4]:
df.head()

,Unnamed: 0,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,LEN,YR,MON,DAY,HR,WK,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,DAY_TYPE_A,DAY_TYPE_B,DAY_TYPE_C,YR_2014
0,0,1372636858620000589,NaN,NaN,20000589,1372636858,330,2013,7,1,0,0,0,0,1,1,0,0,0
1,1,1372637303620000596,NaN,7.0,20000596,1372637303,270,2013,7,1,0,0,0,1,0,1,0,0,0
2,2,1372636951620000320,NaN,NaN,20000320,1372636951,960,2013,7,1,0,0,0,0,1,1,0,0,0
3,3,1372636854620000520,NaN,NaN,20000520,1372636854,630,2013,7,1,0,0,0,0,1,1,0,0,0
4,4,1372637091620000337,NaN,NaN,20000337,1372637091,420,2013,7,1,0,0,0,0,1,1,0,0,0


In [5]:
#Removing these for this MLP model only
df = df.drop(columns=["Unnamed: 0", "TRIP_ID", "ORIGIN_CALL", "ORIGIN_STAND", "TAXI_ID", "TIMESTAMP", "YR_2014", "YR"])

In [6]:
#Take 20% of data to train
df_sample = df.sample(frac=0.2)

In [7]:
df_len = df["LEN"]
df_sample = df_sample.drop(columns=["LEN"])

In [8]:
len(df_sample)

342132

In [9]:
df_sample.head()

,MON,DAY,HR,WK,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,DAY_TYPE_A,DAY_TYPE_B,DAY_TYPE_C
288397,9,5,6,3,1,0,0,1,0,0
221113,8,18,9,6,0,0,1,1,0,0
460317,10,9,8,2,0,1,0,1,0,0
1198467,3,18,13,1,0,1,0,1,0,0
602664,11,7,8,3,0,0,1,1,0,0


In [10]:
features = [df_sample.iloc[i].values for i in range(len(df_sample))]

In [11]:
x_train = torch.tensor(np.array(features), dtype = torch.float).to(device)
y_train = torch.tensor(np.array(df_len.values), dtype = torch.float).to(device)
train_data = list(zip(x_train, y_train))

In [12]:
print(len(train_data))

342132


In [13]:
learning_rate = 1e-5

In [18]:
model = torch.nn.Sequential(
    torch.nn.Linear(10, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 1),
).to(device)

In [19]:
loss_fn = torch.nn.MSELoss().to(device)
#optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [20]:
def train_epoch(train_data, model, optimizer, loss_fn):
    losses = []
    # get a batch of training data
    for x, y in tqdm(train_data):
        # make predictions for this batch
        y_pred = model(x)
        
        # Compute the RSME loss
        loss = torch.sqrt(loss_fn(y_pred, y))
        
        # Backpropagation
        # zero out the gradients so that it will not accumulate through each iteration
        optimizer.zero_grad()
        
        # Compute the gradents with the backward call (backprop)
        loss.backward()
        
        # Update weight using gradient descent 
        optimizer.step()
        
        losses.append(loss.item())

    return np.mean(losses)

def eval_epoch(valid_loader, model, loss_function):
    total = 0
    correct = 0
    preds = []
    trues = []
    with torch.no_grad(): 
        for x, t in valid_loader:
            # Compute prediction
            y = model(x)
            
            total += 1
            correct += ((y <= t+10) and (y >= t-10))
            #trues.append(t)
            preds.append(y)
            
    return float(correct/total*100), preds

In [21]:
total_epochs = 5
losses = []
train_accs, valid_accs = [], []
max_acc = 0
for epoch in range(total_epochs):
    
    model.train() # gradient tracking is on
    
    train_loss = train_epoch(train_data, model, optimizer, loss_fn)
    
    losses.append(train_loss)
    
    model.eval()
    train_acc, train_pred = eval_epoch(train_data[:100], model, loss_fn)
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss:>0.4f}, Train Accuracy {train_acc:>0.4f}%\n")

100%|██████████| 342132/342132 [08:18<00:00, 685.76it/s]


Epoch: 1, Train Loss: 334.1468, Train Accuracy 0.0000%



100%|██████████| 342132/342132 [08:21<00:00, 681.93it/s]


Epoch: 2, Train Loss: 317.6744, Train Accuracy 1.0000%



100%|██████████| 342132/342132 [08:18<00:00, 686.53it/s]


Epoch: 3, Train Loss: 316.9215, Train Accuracy 3.0000%



100%|██████████| 342132/342132 [08:17<00:00, 687.74it/s]


Epoch: 4, Train Loss: 316.6203, Train Accuracy 2.0000%



100%|██████████| 342132/342132 [08:40<00:00, 657.43it/s]


Epoch: 5, Train Loss: 316.4255, Train Accuracy 2.0000%



In [25]:
torch.save(model, "MLP")

In [26]:
df_test = pd.read_csv("processed_test.csv")
df_out = df_test["TRIP_ID"].to_frame()
df_test = df_test.drop(columns=["Unnamed: 0", "TRIP_ID", "ORIGIN_CALL", "ORIGIN_STAND", "TAXI_ID", "TIMESTAMP", "YR_2014", "YR"])

In [27]:
x_test = [torch.tensor(df_test.iloc[i].values,dtype=torch.float).to(device) for i in range(len(df_test))]

In [24]:
model.eval()
with torch.no_grad(): 
    df_out["TRAVEL_TIME"] = [float(model(x).cpu()) for x in x_test]
df_out.head()
# # mean(716.43) -> 792.73593
# # median(600) -> 784.74219
df_out.to_csv("my_pred.csv", index=None)

In [ ]:
#TODO Implement accuracy or smthing for training
#Do something with validation (make a validation set)